# Exercícios

1. Baixar o arquivo "Lena.png" e aprender a carregá-la em um array 2D numpy

2. Implementar a transformada DFT em 2D

3. Comparar resultados com a função numpy.fft.fft2(.)

4. Transformar "Lena" para o espaço de frequências e plotar seu espectro

5. Utilizar uma função Gaussiana centralizada no meio da imagem e com desvios padrão sigma_x e sigma_y, e filtrar o espectro

6. Realizar a transformada inversa após filtragem do espectro

7. Criar um array 2D em numpy que represente um fundo preto e um quadrado "branco" de dimensões 1/4 da imagem, posicionado no centro da imagem, com intensidade=1.

8. Transformar a imagem para o espaço de frequências e filtrá-la com a mesma função Gaussiana

9. Observar o efeito da filtragem no espaço da imagem, com diferentes sigmas.

10. Repetir os passos 7-9, rotacionando o quadro em 30 graus. (Dica: veja a função rotate da biblioteca scipy)

In [1]:
import matplotlib as mpl
import matplotlib.cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from matplotlib import colorbar
from matplotlib.colors import LogNorm, Normalize, to_rgb

from mpl_toolkits.axes_grid1 import make_axes_locatable

from PIL import Image

mpl.rcParams['axes.titlesize'] = 10
mpl.rcParams['axes.labelsize'] = 8
mpl.rcParams['xtick.labelsize'] = 8
mpl.rcParams['ytick.labelsize'] = 8

In [2]:
def add_colorbar(ax, pos='right', size=0.1, pad=0.05, 
                 cmap=None, norm=None, off=False, 
                 orientation='vertical', sharex=None):
    divider = make_axes_locatable(ax)
    bar = divider.append_axes(pos, size, pad=pad, sharex=sharex)
    
    if isinstance(cmap, str):
        cmap = matplotlib.cm.cmap_d[cmap]

    if off:
        bar.axis('off')
    else:
        colorbar.ColorbarBase(bar, cmap=cmap, norm=norm, 
                              orientation=orientation)

    return bar

In [3]:
img = np.zeros((512, 512))

In [4]:
def drawimg(ax, title, img, cmap, norm):
    ax.set_title(title)
    ax.imshow(img, cmap=cmap, norm=norm)
    add_colorbar(ax, cmap=cmap, norm=norm)

In [5]:
def gauss(arr, sx, sy):
    X, Y = arr.shape
    x, y = np.mgrid[0:X, 0:Y]
    
    return np.exp(-((x-X/2)**2/(2*sx**2) + (y-Y/2)**2/(2*sy**2))**2)

In [6]:
from ipywidgets import interactive
from scipy import ndimage

def plot(cx=1/2, cy=1/2, ax=1/5, ay=1/5, sx=0.5, sy=0.5, deg=0):
    fig, axis = plt.subplots(2, 4, figsize=(24, 12))
    fig.set_tight_layout(True)
    
    s = img.shape
    
    ret = np.zeros(img.shape)
    ret[max(int(s[0]*cx-s[0]*ax/2), 0):min(int(s[0]*cx+s[0]*ax/2), s[1]), \
        max(int(s[1]*cy-s[1]*ay/2), 0):min(int(s[1]*cy+s[1]*ay/2), s[1])] = 1
    ret = ndimage.rotate(ret, deg, reshape=False, order=5)
    
    ret_fft = np.fft.fftshift(np.fft.fft2(ret))
    
    norml0 = Normalize(0, 1)
    normlp = Normalize(0, np.log1p(np.sum(ret)))
    
    drawimg(axis[0, 0], 'Rectangle Filter', ret, 'gray', norml0)
    drawimg(axis[0, 1], 'SFFT2 Rectangle Filter Abs', np.log1p(np.abs(ret_fft)), 'viridis', normlp)
    drawimg(axis[0, 2], 'SFFT2 Rectangle Filter Real', np.log1p(np.real(ret_fft)), 'viridis', normlp)
    drawimg(axis[0, 3], 'SFFT2 Rectangle Filter Imag', np.log1p(np.imag(ret_fft)), 'viridis', normlp)
    
    gss = gauss(ret, 1/sx, 1/sy)
    ret_gss = np.real(np.fft.ifft2(np.fft.ifftshift(np.fft.fftshift(np.fft.fft2(ret))*gss)))
    
    ret_gss_fft = np.fft.fftshift(np.fft.fft2(ret_gss))
    
    drawimg(axis[1, 0], 'Rectangle Gaussian-Filtered', ret_gss, 'gray', norml0)
    drawimg(axis[1, 1], 'SFFT2 Rectangle Gaussian-Filtered Abs', np.log1p(np.abs(ret_gss_fft)), 'viridis', normlp)
    drawimg(axis[1, 2], 'SFFT2 Rectangle Gaussian-Filtered Real', np.log1p(np.real(ret_gss_fft)), 'viridis', normlp)
    drawimg(axis[1, 3], 'SFFT2 Rectangle Gaussian-Filtered Imag', np.log1p(np.imag(ret_gss_fft)), 'viridis', normlp)

interactive(plot, cx=(0, 1, 0.1), cy=(0, 1, 0.1), ax=(0, 1, 0.1), ay=(0, 1, 0.1), sx=(0.01, 0.1, 0.01), sy=(0.01, 0.1, 0.01), deg=(0, 360, 10))

interactive(children=(FloatSlider(value=0.5, description='cx', max=1.0), FloatSlider(value=0.5, description='c…